In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import os
import cv2

In [2]:
prototxtPath = os.path.sep.join([r'C:\Users\Dostam\FaceMask Detection\face_detector','deploy.prototxt'])
weightsPath = os.path.sep.join([r'C:\Users\Dostam\FaceMask Detection\face_detector','res10_300x300_ssd_iter_140000.caffemodel'])


In [3]:
net = cv2.dnn.readNet(prototxtPath,weightsPath)

In [4]:
model = load_model(r'C:\Users\Dostam\FaceMask Detection\mask_detector.model')

In [5]:
image = cv2.imread(r'C:\Users\Dostam\FaceMask Detection\examples\db3.jpg')

In [6]:
image

array([[[ 66,  77,  81],
        [ 66,  77,  81],
        [ 66,  77,  81],
        ...,
        [ 48,  41,  48],
        [ 48,  41,  48],
        [ 48,  41,  48]],

       [[ 66,  77,  81],
        [ 66,  77,  81],
        [ 66,  77,  81],
        ...,
        [ 48,  41,  48],
        [ 48,  41,  48],
        [ 48,  41,  48]],

       [[ 66,  77,  81],
        [ 66,  77,  81],
        [ 66,  77,  81],
        ...,
        [ 48,  41,  48],
        [ 48,  41,  48],
        [ 48,  41,  48]],

       ...,

       [[ 30,  34,  28],
        [ 30,  34,  28],
        [ 32,  34,  28],
        ...,
        [155, 166, 174],
        [155, 166, 174],
        [155, 166, 174]],

       [[ 30,  34,  28],
        [ 30,  34,  28],
        [ 32,  34,  28],
        ...,
        [155, 166, 174],
        [155, 166, 174],
        [155, 166, 174]],

       [[ 30,  34,  28],
        [ 30,  34,  28],
        [ 32,  34,  28],
        ...,
        [156, 167, 175],
        [156, 167, 175],
        [156, 167, 175]]

In [7]:
image.shape

(1080, 540, 3)

In [8]:
(h,w)=image.shape[:2]

In [9]:
(h,w)

(1080, 540)

In [10]:
blob = cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [11]:
blob

array([[[[ -38.,  -38.,  -38., ...,  -57.,  -56.,  -56.],
         [ -36.,  -36.,  -36., ...,  -57.,  -56.,  -56.],
         [ -36.,  -36.,  -36., ...,  -56.,  -56.,  -56.],
         ...,
         [ -72.,  -72.,  -70., ...,   50.,   48.,   48.],
         [ -74.,  -72.,  -70., ...,   51.,   49.,   49.],
         [ -74.,  -72.,  -70., ...,   54.,   51.,   51.]],

        [[-100., -100., -100., ..., -137., -136., -136.],
         [ -98.,  -98.,  -98., ..., -137., -136., -136.],
         [ -98.,  -98.,  -98., ..., -136., -136., -136.],
         ...,
         [-145., -145., -146., ...,  -12.,  -14.,  -14.],
         [-143., -143., -144., ...,  -11.,  -13.,  -13.],
         [-143., -143., -144., ...,   -8.,  -11.,  -11.]],

        [[ -42.,  -42.,  -42., ...,  -76.,  -75.,  -75.],
         [ -40.,  -40.,  -40., ...,  -76.,  -75.,  -75.],
         [ -40.,  -40.,  -40., ...,  -75.,  -75.,  -75.],
         ...,
         [ -97.,  -97.,  -96., ...,   50.,   48.,   48.],
         [ -95.,  -95.,  -

In [12]:
blob.shape

(1, 3, 300, 300)

In [13]:
net.setInput(blob)
detections = net.forward()

In [14]:
detections

array([[[[0.        , 1.        , 0.994072  , ..., 0.05900285,
          0.46655366, 0.27423498],
         [0.        , 1.        , 0.12479814, ..., 0.13185099,
          4.972652  , 0.8840864 ],
         [0.        , 1.        , 0.12101994, ..., 4.0047607 ,
          4.841434  , 4.9820366 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [15]:
for i in range(0,detections.shape[2]):
    confidence = detections[0,0,i,2]
    
    if confidence>0.5:
        box = detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY) = box.astype('int')
        
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        label = 'Mask' if mask > withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()